# Lexical Analyser

Owner: André Geraldo 

Course: Computer Enginnering

Steps:

1- Read a file containing a source code with a specific syntax

2 - User a buffer to take each line and classify them 

3 - Regex will recognise if a token is valid and store them

4 - store each token in the the following format:
 [Token, Lexema, Line, Column]

In [44]:
from typing import NamedTuple
import re

class Token(NamedTuple):
  type: str
  lexema: str
  line: int
  col: int

class LexicalAnalyser:

  def __init__(self, source_code):
    self.__keywords = {
        'BeginFun','EndFun','if','then','else','elif',
        'end','funLoopWhile','do','endFunLoop',
        'showMeTheCode','grabInput','funny'
        }
    self.__token_pair = [
        ('TK_NUM', r'\d+'),
        ('TK_ATRIB', r'<-'),
        ('TK_PERIOD', r'\.'),
        ('TK_ID', r'[A-Za-z]([A-Za-z]|\d|_)*'),
        ('TK_STRING', r'^\"([A-Za-z]|\d|\.|%|:| )*\"$'), #String com aspas
        ('TK_OP_AR', r'[-+*\/]'), #Operações aritmética
        ('TK_OP_RE', r'<>|[=<>]'),#Operação relacional  
        ('TK_BOOL', r'[|&]'),       
        ('TK_OPEN_P', r'\('),
        ('TK_CLOSE_P', r'\)'),
        ('TK_COMMA', r','),
        ('TK_NEW_LINE',r'\n'),
        ('TK_SKIP', r'[\ \t\r]+'),
        ('MISMATCH', r'.'),                
      ]                            
    self.__source_code = source_code

  def generateToken(self):
    
    token_base = self.__token_pair
    buffer = self.__source_code
    #Cria grupos com o nome de cada par de token no formato: (?P<nomeTk>regex)
    #Concatena cada par com '|' que vai servir como OU em passos seguintes
    regex_rules = '|'.join('(?P<%s>%s)' % strPair for strPair in token_base)
    line = 1
    col_start = 0

    #tkType: Pega o nome do padrão reconhecido mais recente. Dependendo 
    #do padrão existem algumas correções antes de salvar como um token
    #lexema: pega o valor extraído quando o padrão é reconhecido
    #column: calcula o valor inicial da coluno do token encontrado

    for matchedPattern in re.finditer(regex_rules, buffer):
      tkType = matchedPattern.lastgroup
      lexema = matchedPattern.group()
      column = matchedPattern.start()+1 - col_start
      #print("LASTGROUP: ", kind)
      #print("\nVALUE: ", mo.group())
      #print("\nCOLLUMN: ", mo.start() - line_start)
      if tkType == 'TK_NUM':
        lexema = int(lexema)
      elif lexema in self.__keywords :
        tkType = lexema
      elif tkType == 'TK_NEW_LINE':
        col_start = matchedPattern.end()
        line += 1
        continue
      elif tkType == 'TK_SKIP':
        continue
      elif tkType == 'MISMATCH':
        raise RuntimeError(f'{tkType!r} valor inesperado na linha {line}')
      #Retorna uma função geradora a quem chamou. A execução começa
      #apenas quando o gerador é iterado
      #Vantagem: Nenhuma memória é alocada quando o yield é usado
      #O parâmetro é uma tupla nomeada (NamedTuple)
      yield Token(tkType,lexema,line, column)
    
def main():
  #read the current fun lang file in test.lang
  # and store it in a buffer

  content = ""
  with open("lang.txt", "r") as file:
    content = file.read()

  lexico = LexicalAnalyser(content).generateToken()


  for token in lexico:
    print(token)
    print('\n')

main()

Token(type='BeginFun', lexema='BeginFun', line=1, col=1)


Token(type='if', lexema='if', line=2, col=2)


Token(type='TK_ID', lexema='idade', line=2, col=5)


Token(type='TK_OP_RE', lexema='<', line=2, col=11)


Token(type='TK_NUM', lexema=10, line=2, col=13)


Token(type='TK_BOOL', lexema='&', line=2, col=16)


Token(type='TK_ID', lexema='anoNasc', line=2, col=18)


Token(type='TK_OP_RE', lexema='<>', line=2, col=26)


Token(type='TK_NUM', lexema=10, line=2, col=29)


Token(type='then', lexema='then', line=2, col=32)


Token(type='funLoopWhile', lexema='funLoopWhile', line=3, col=2)


Token(type='TK_ID', lexema='valor_ethereum', line=3, col=15)


Token(type='TK_OP_RE', lexema='<', line=3, col=30)


Token(type='TK_ID', lexema='valor_bitcoin', line=3, col=32)


Token(type='do', lexema='do', line=3, col=46)


Token(type='showMeTheCode', lexema='showMeTheCode', line=4, col=3)


Token(type='TK_ID', lexema='investYourMoney', line=4, col=17)


Token(type='TK_PERIOD', lexema='.', line=4, col=